In [4]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from scipy import stats
import numpy as np
import seaborn as sns
import xgboost as xgb
from sklearn.feature_selection import mutual_info_regression
from sklearn.feature_selection import f_regression
from sklearn.feature_selection import RFE
# from tsfresh import 
from sklearn.metrics import mean_squared_error

In [5]:
train_targets = pd.read_parquet('./data/A/train_targets.parquet')
X_test_estimated = pd.read_parquet('./data/A/X_test_estimated.parquet')
X_train_estimated = pd.read_parquet('./data/A/X_train_estimated.parquet')
X_train_observed = pd.read_parquet('./data/A/X_train_observed.parquet')

In [ ]:
df = pd.concat([X_train_observed, X_train_estimated])
df = pd.merge(df, train_targets, left_on='date_forecast', right_on='time', how='inner')
df = df.drop(columns=['date_calc', 'date_forecast'])
df = df.drop(columns=['elevation:m', 'snow_density:kgm3', 'snow_drift:idx'], axis=1)

imputer = SimpleImputer(strategy='mean')
df[['ceiling_height_agl:m', 'cloud_base_agl:m']] = imputer.fit_transform(df[['ceiling_height_agl:m', 'cloud_base_agl:m']])

X = df.drop(columns=['pv_measurement', 'time'])  # Drop the index, target, and time columns
y = df['pv_measurement']  # Target variable

In [ ]:
# Sorting the data by the 'time' column to maintain chronological order
df.sort_values('time', inplace=True)

# Identifying the features and the target variable
features = df.drop(columns=['pv_measurement', 'time'])  # Assuming all other columns are features
target = df['pv_measurement']

# Splitting the data into training and test sets
split_point = int(len(df) * 0.8)  # Taking 80% of the data as the training set
X_train = features.iloc[:split_point]
y_train = target.iloc[:split_point]
X_test = features.iloc[split_point:]
y_test = target.iloc[split_point:]

# Displaying the shapes of the training and test sets
X_train.shape, X_test.shape, y_train.shape, y_test.shape

# First Baseline Model: XGBoost

In [ ]:
xgb_model = xgb.XGBRegressor()
xgb_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = xgb_model.predict(X_test)

# Calculate MSE on the test set
mse = mean_squared_error(y_test, y_pred)
mse